In [1]:
import awswrangler as awr
import pandas as pd
import os
import openpyxl

In [22]:
caminho_query = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\sql\faturas_inadimplentes.sql"

#realizando a leitura da query
with open(caminho_query,'r') as arquivo_query:
    query = arquivo_query.read()

#transformando em dataframe (pandas) e executando a consulta no athena 
df_inadimp = awr.athena.read_sql_query(query, database='silver')



In [25]:
boletos_pagos = df_inadimp.loc[df_inadimp['historico']!=1,'ponteiro'].tolist()
boletos_pagos


[188784,
 198649,
 200262,
 206438,
 206516,
 219234,
 219397,
 230871,
 225944,
 227684,
 230119,
 235134,
 215025,
 220124,
 234040,
 154408,
 164665,
 164867,
 176849,
 177710,
 201452,
 207989,
 214012,
 214695,
 231844,
 214325,
 187579,
 226248,
 214183,
 220812,
 155638,
 167203,
 179637,
 179888,
 181728,
 188201,
 190076,
 196787,
 203207,
 203169,
 210667,
 214481,
 215427,
 220595,
 220946,
 231177,
 231536,
 236048,
 236766,
 236869,
 170034,
 170021,
 149264,
 158929,
 159418,
 162176,
 163760,
 164422,
 165402,
 166617,
 169925,
 169247,
 159854,
 194661,
 194660,
 195870,
 192334,
 194047,
 157960,
 194154,
 177799,
 175300,
 191759,
 149924,
 149928,
 151557,
 191422,
 182508,
 162369,
 165405,
 166229,
 166521,
 170381,
 172570,
 175408,
 195178,
 159635,
 192002,
 172952,
 194308,
 168771,
 197116,
 173860,
 195083,
 198903,
 191201,
 169378,
 199596,
 172195,
 199502,
 199504,
 149723,
 150755,
 151254,
 151334,
 162996,
 173820,
 174685,
 183889,
 198735,
 181962,
 

In [6]:
#ponteiros cujo historico é diferente de 1
lista_boletos_pagos = df_inadimp.loc[df_inadimp['historico'] != 1, ['ponteiro','conjunto','matricula']].values.tolist()
lista_boletos_pagos


[[178815, <NA>, <NA>],
 [195468, <NA>, <NA>],
 [195871, <NA>, <NA>],
 [201758, <NA>, <NA>],
 [188349, <NA>, <NA>],
 [173274, <NA>, <NA>],
 [202590, <NA>, <NA>],
 [199360, <NA>, <NA>],
 [188015, <NA>, <NA>],
 [169191, <NA>, <NA>],
 [157571, <NA>, <NA>],
 [160711, <NA>, <NA>],
 [163747, <NA>, <NA>],
 [163757, <NA>, <NA>],
 [150728, <NA>, <NA>],
 [151825, <NA>, <NA>],
 [153918, <NA>, <NA>],
 [161484, <NA>, <NA>],
 [161591, <NA>, <NA>],
 [162446, <NA>, <NA>],
 [163508, <NA>, <NA>],
 [163515, <NA>, <NA>],
 [163517, <NA>, <NA>],
 [163519, <NA>, <NA>],
 [162876, <NA>, <NA>],
 [163058, <NA>, <NA>],
 [155720, <NA>, <NA>],
 [161172, <NA>, <NA>],
 [153302, <NA>, <NA>],
 [156580, <NA>, <NA>],
 [163975, <NA>, <NA>],
 [164668, <NA>, <NA>],
 [153593, <NA>, <NA>],
 [165256, <NA>, <NA>],
 [155570, <NA>, <NA>],
 [156701, <NA>, <NA>],
 [157422, <NA>, <NA>],
 [160493, <NA>, <NA>],
 [185509, <NA>, <NA>],
 [185892, <NA>, <NA>],
 [186452, <NA>, <NA>],
 [190833, <NA>, <NA>],
 [194000, <NA>, <NA>],
 [197104, <

In [23]:
df_inadimp['historico'].unique()

<IntegerArray>
[1, 8, 3, 4, 7]
Length: 5, dtype: Int64

In [26]:
# Gerando um index para comparação de séries
#df_inadimp = df_inadimp.set_index(['ponteiro', 'conjunto', 'matricula'])
# lista_boletos_pagos = lista_boletos_pagos.set_index(['ponteiro', 'conjunto', 'matricula'])

# Comparando os index e filtrando
#df_inadimp = df_inadimp[~df_inadimp.index.isin(lista_boletos_pagos.index)]

df_inadimp = df_inadimp[~df_inadimp['ponteiro'].isin(boletos_pagos)]

# Resetando o index para manter o padrão do restante do código
# df_inadimp = df_inadimp.reset_index()



In [27]:
#filtrando com status de a pagar
df_inadimp = df_inadimp[df_inadimp['historico']==1]


df_inadimp = df_inadimp.drop_duplicates(subset=['ponteiro', 'conjunto', 'matricula', 'empresa'], keep='first')

In [28]:
caminho_pasta = r'C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Inadimplência'
caminho_arquivo = os.path.join(caminho_pasta,'relatorio_inadimplencia.xlsx')

#verificando a existência da pasta e removendo a versão antiga
os.makedirs(caminho_pasta,exist_ok=True)
if os.path.exists(caminho_arquivo):
    os.remove(caminho_arquivo)
    print("Arquivo antigo removido, iniciando carregamento...")

#convertendo o dataframe em excel e associando ao caminho da pasta
df_inadimp.to_excel(caminho_arquivo, engine = 'openpyxl', index=False, sheet_name='inadimplentes')

print(f"Arquivo Excel salvo com sucesso em: {caminho_arquivo}")

Arquivo antigo removido, iniciando carregamento...
Arquivo Excel salvo com sucesso em: C:\Users\raphael.almeida\OneDrive - Grupo Unus\analise de dados - Arquivos em excel\Relatório de Inadimplência\relatorio_inadimplencia.xlsx
